## Data Access

In [0]:

spark.conf.set("fs.azure.account.auth.type..dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type..dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id..dfs.core.windows.net", "")
spark.conf.set("fs.azure.account.oauth2.client.secret..dfs.core.windows.net", "")
spark.conf.set("fs.azure.account.oauth2.client.endpoint..dfs.core.windows.net", "https://login.microsoftonline.com//oauth2/token")
# Azure OAuth configuration - Credentials stored securely in Databricks secrets
# Actual client_id, client_secret and tenant_id removed for security

In [0]:
dbutils.fs.ls('abfss://bronze@nyctaxidataproject1.dfs.core.windows.net')

[FileInfo(path='abfss://bronze@nyctaxidataproject1.dfs.core.windows.net/trip_type/', name='trip_type/', size=0, modificationTime=1764709899000),
 FileInfo(path='abfss://bronze@nyctaxidataproject1.dfs.core.windows.net/trip_zone/', name='trip_zone/', size=0, modificationTime=1764709908000),
 FileInfo(path='abfss://bronze@nyctaxidataproject1.dfs.core.windows.net/tripsdata_2024/', name='tripsdata_2024/', size=0, modificationTime=1764718632000)]

## Data Reading

**Importing Libraries**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Reading CSV Data

**Trip Type Data**

In [0]:
df_trip_type = spark.read.format("csv")\
                    .option("header",True)\
                    .option("inferSchema",True)\
                    .load("abfss://bronze@nyctaxidataproject1.dfs.core.windows.net/trip_type")

In [0]:
df_trip_type.display()

trip_type,description
1,Street-hail
2,Dispatch


**Trip Zone Data**

In [0]:
df_trip_zone = spark.read.format("csv")\
                    .option("header",True)\
                    .option("inferSchema",True)\
                    .load("abfss://bronze@nyctaxidataproject1.dfs.core.windows.net/trip_zone")

In [0]:
df_trip_zone.limit(20).display()
print(f"Total rows: {df_trip_zone.count():,}")

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
7,Queens,Astoria,Boro Zone
8,Queens,Astoria Park,Boro Zone
9,Queens,Auburndale,Boro Zone
10,Queens,Baisley Park,Boro Zone


Total rows: 265


**Trip Data**

In [0]:
df_trip = spark.read.format("parquet")\
               .option("inferSchema",True)\
               .option("header",True)\
               .option("recursiveFileLookup",True)\
                .load("abfss://bronze@nyctaxidataproject1.dfs.core.windows.net/tripsdata_2024")
               

In [0]:
df_trip.limit(10).display()

print(f"Total rows: {df_trip.count():,}")

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
2,2024-10-01T00:30:44,2024-10-01T00:48:26,1,3.0,1,N,162,246,1,18.4,1.0,0.5,1.5,0.0,1.0,24.9,2.5,0.0
1,2024-10-01T00:12:20,2024-10-01T00:25:25,1,2.2,1,N,48,236,1,14.2,3.5,0.5,3.8,0.0,1.0,23.0,2.5,0.0
1,2024-10-01T00:04:46,2024-10-01T00:13:52,1,2.7,1,N,142,24,1,13.5,3.5,0.5,3.7,0.0,1.0,22.2,2.5,0.0
1,2024-10-01T00:12:10,2024-10-01T00:23:01,1,3.1,1,N,233,75,1,14.2,3.5,0.5,2.0,0.0,1.0,21.2,2.5,0.0
1,2024-10-01T00:30:22,2024-10-01T00:30:39,1,0.0,1,N,262,262,3,3.0,3.5,0.5,0.0,0.0,1.0,8.0,2.5,0.0
2,2024-10-01T00:31:20,2024-10-01T00:36:00,2,0.97,1,N,137,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.0
1,2024-10-01T00:42:57,2024-10-01T00:49:01,1,1.3,1,N,142,48,1,7.9,3.5,0.5,2.55,0.0,1.0,15.45,2.5,0.0
1,2024-10-01T00:59:55,2024-10-01T01:02:24,1,0.5,1,N,230,161,1,5.1,3.5,0.5,2.0,0.0,1.0,12.1,2.5,0.0
1,2024-10-01T00:00:47,2024-10-01T00:04:22,0,1.1,1,N,142,237,1,7.2,3.5,0.5,3.0,0.0,1.0,15.2,2.5,0.0
1,2024-10-01T00:17:36,2024-10-01T00:26:22,1,2.2,1,N,162,145,1,11.4,3.5,0.5,3.3,0.0,1.0,19.7,2.5,0.0


Total rows: 41,169,720


## Data Transformation

**Taxi trip type**

In [0]:
df_trip_type.display()

trip_type,description
1,Street-hail
2,Dispatch


In [0]:
df_trip_type = df_trip_type.withColumnRenamed('description','trip_description')
df_trip_type.display()

trip_type,trip_description
1,Street-hail
2,Dispatch


In [0]:
df_trip_type.write.format('parquet')\
                  .mode('overwrite')\
                  .option("path","abfss://silver@nyctaxidataproject1.dfs.core.windows.net/trip_type")\
                  .save()

**Trip Zone**

In [0]:
df_trip_zone.limit(10).display()

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
7,Queens,Astoria,Boro Zone
8,Queens,Astoria Park,Boro Zone
9,Queens,Auburndale,Boro Zone
10,Queens,Baisley Park,Boro Zone


In [0]:
# Split zone names that have '/' into two separate columns
df_trip_zone = df_trip_zone.withColumn('zone1', split(col('Zone'), '/')[0])\
                           .withColumn('zone2', split(col('Zone'), '/')[1])
df_trip_zone.limit(10).display()

LocationID,Borough,Zone,service_zone,zone1,zone2
1,EWR,Newark Airport,EWR,Newark Airport,null
2,Queens,Jamaica Bay,Boro Zone,Jamaica Bay,null
3,Bronx,Allerton/Pelham Gardens,Boro Zone,Allerton,Pelham Gardens
4,Manhattan,Alphabet City,Yellow Zone,Alphabet City,null
5,Staten Island,Arden Heights,Boro Zone,Arden Heights,null
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone,Arrochar,Fort Wadsworth
7,Queens,Astoria,Boro Zone,Astoria,null
8,Queens,Astoria Park,Boro Zone,Astoria Park,null
9,Queens,Auburndale,Boro Zone,Auburndale,null
10,Queens,Baisley Park,Boro Zone,Baisley Park,null


In [0]:
# Check what unique borough values we have in the data
df_trip_zone.select("Borough").distinct().display()

Borough
Queens
EWR
Unknown
Brooklyn
Staten Island
N/A
Manhattan
Bronx


In [0]:
# CREATE STANDARDIZED BOROUGH COLUMN
# Preserve original 'Borough' column, create cleaned version
df_trip_zone = df_trip_zone.withColumn(
    "standardized_borough",
    when(col("Borough") == "EWR", "Newark Airport")           # EWR is Newark Airport
    .when(col("Borough") == "N/A", None)                      # Invalid → NULL
    .when(col("Borough") == "Unknown", None)                  # Invalid → NULL
    .otherwise(col("Borough"))                                # Keep valid values as-is
)

# CREATE DATA QUALITY FLAG
# Flag records with valid borough information
df_trip_zone = df_trip_zone.withColumn(
    "is_valid_borough",
    when(col("standardized_borough").isNull(), False)         # NULL = Invalid
    .when(col("standardized_borough").isin([
        "Manhattan", "Brooklyn", "Queens", 
        "Bronx", "Staten Island", "Newark Airport"
    ]), True)                                                 # Valid boroughs
    .otherwise(False)                                         # Any other = Invalid
)

df_trip_zone.limit(10).display()

LocationID,Borough,Zone,service_zone,zone1,zone2,standardized_borough,is_valid_borough
1,EWR,Newark Airport,EWR,Newark Airport,null,Newark Airport,true
2,Queens,Jamaica Bay,Boro Zone,Jamaica Bay,null,Queens,true
3,Bronx,Allerton/Pelham Gardens,Boro Zone,Allerton,Pelham Gardens,Bronx,true
4,Manhattan,Alphabet City,Yellow Zone,Alphabet City,null,Manhattan,true
5,Staten Island,Arden Heights,Boro Zone,Arden Heights,null,Staten Island,true
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone,Arrochar,Fort Wadsworth,Staten Island,true
7,Queens,Astoria,Boro Zone,Astoria,null,Queens,true
8,Queens,Astoria Park,Boro Zone,Astoria Park,null,Queens,true
9,Queens,Auburndale,Boro Zone,Auburndale,null,Queens,true
10,Queens,Baisley Park,Boro Zone,Baisley Park,null,Queens,true


In [0]:
# Keep all records, but tag invalid ones
df_trip_zone = df_trip_zone.withColumn(
    "data_quality_issue",
    when(col("is_valid_borough") == False, "Invalid Borough").otherwise(None)
)
df_trip_zone.limit(10).display()

LocationID,Borough,Zone,service_zone,zone1,zone2,standardized_borough,is_valid_borough,data_quality_issue
1,EWR,Newark Airport,EWR,Newark Airport,null,Newark Airport,true,null
2,Queens,Jamaica Bay,Boro Zone,Jamaica Bay,null,Queens,true,null
3,Bronx,Allerton/Pelham Gardens,Boro Zone,Allerton,Pelham Gardens,Bronx,true,null
4,Manhattan,Alphabet City,Yellow Zone,Alphabet City,null,Manhattan,true,null
5,Staten Island,Arden Heights,Boro Zone,Arden Heights,null,Staten Island,true,null
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone,Arrochar,Fort Wadsworth,Staten Island,true,null
7,Queens,Astoria,Boro Zone,Astoria,null,Queens,true,null
8,Queens,Astoria Park,Boro Zone,Astoria Park,null,Queens,true,null
9,Queens,Auburndale,Boro Zone,Auburndale,null,Queens,true,null
10,Queens,Baisley Park,Boro Zone,Baisley Park,null,Queens,true,null


In [0]:
df_trip_zone.write.format('parquet')\
                  .mode('overwrite')\
                  .option("path","abfss://silver@nyctaxidataproject1.dfs.core.windows.net/trip_zone")\
                  .save()

**Trip Data**

In [0]:
df_trip.limit(15).display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
2,2024-10-01T00:30:44,2024-10-01T00:48:26,1,3.0,1,N,162,246,1,18.4,1.0,0.5,1.5,0.0,1.0,24.9,2.5,0.0
1,2024-10-01T00:12:20,2024-10-01T00:25:25,1,2.2,1,N,48,236,1,14.2,3.5,0.5,3.8,0.0,1.0,23.0,2.5,0.0
1,2024-10-01T00:04:46,2024-10-01T00:13:52,1,2.7,1,N,142,24,1,13.5,3.5,0.5,3.7,0.0,1.0,22.2,2.5,0.0
1,2024-10-01T00:12:10,2024-10-01T00:23:01,1,3.1,1,N,233,75,1,14.2,3.5,0.5,2.0,0.0,1.0,21.2,2.5,0.0
1,2024-10-01T00:30:22,2024-10-01T00:30:39,1,0.0,1,N,262,262,3,3.0,3.5,0.5,0.0,0.0,1.0,8.0,2.5,0.0
2,2024-10-01T00:31:20,2024-10-01T00:36:00,2,0.97,1,N,137,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.0
1,2024-10-01T00:42:57,2024-10-01T00:49:01,1,1.3,1,N,142,48,1,7.9,3.5,0.5,2.55,0.0,1.0,15.45,2.5,0.0
1,2024-10-01T00:59:55,2024-10-01T01:02:24,1,0.5,1,N,230,161,1,5.1,3.5,0.5,2.0,0.0,1.0,12.1,2.5,0.0
1,2024-10-01T00:00:47,2024-10-01T00:04:22,0,1.1,1,N,142,237,1,7.2,3.5,0.5,3.0,0.0,1.0,15.2,2.5,0.0
1,2024-10-01T00:17:36,2024-10-01T00:26:22,1,2.2,1,N,162,145,1,11.4,3.5,0.5,3.3,0.0,1.0,19.7,2.5,0.0


In [0]:
# Split datetime into date, year, month columns for easier analysis
df_trip = df_trip.withColumn('trip_pickup_date',to_date(col('tpep_pickup_datetime')))\
                 .withColumn('trip_pickup_year',year(col('tpep_pickup_datetime')))\
                 .withColumn('trip_pickup_month',month(col('tpep_pickup_datetime')))\
                 .withColumn('trip_dropoff_date',to_date(col('tpep_dropoff_datetime')))\
                 .withColumn('trip_dropoff_year',year(col('tpep_dropoff_datetime')))\
                 .withColumn('trip_dropoff_month',month(col('tpep_dropoff_datetime')))
df_trip.limit(15).display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_pickup_date,trip_pickup_year,trip_pickup_month,trip_dropoff_date,trip_dropoff_year,trip_dropoff_month
2,2024-10-01T00:30:44,2024-10-01T00:48:26,1,3.0,1,N,162,246,1,18.4,1.0,0.5,1.5,0.0,1.0,24.9,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10
1,2024-10-01T00:12:20,2024-10-01T00:25:25,1,2.2,1,N,48,236,1,14.2,3.5,0.5,3.8,0.0,1.0,23.0,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10
1,2024-10-01T00:04:46,2024-10-01T00:13:52,1,2.7,1,N,142,24,1,13.5,3.5,0.5,3.7,0.0,1.0,22.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10
1,2024-10-01T00:12:10,2024-10-01T00:23:01,1,3.1,1,N,233,75,1,14.2,3.5,0.5,2.0,0.0,1.0,21.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10
1,2024-10-01T00:30:22,2024-10-01T00:30:39,1,0.0,1,N,262,262,3,3.0,3.5,0.5,0.0,0.0,1.0,8.0,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10
2,2024-10-01T00:31:20,2024-10-01T00:36:00,2,0.97,1,N,137,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10
1,2024-10-01T00:42:57,2024-10-01T00:49:01,1,1.3,1,N,142,48,1,7.9,3.5,0.5,2.55,0.0,1.0,15.45,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10
1,2024-10-01T00:59:55,2024-10-01T01:02:24,1,0.5,1,N,230,161,1,5.1,3.5,0.5,2.0,0.0,1.0,12.1,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10
1,2024-10-01T00:00:47,2024-10-01T00:04:22,0,1.1,1,N,142,237,1,7.2,3.5,0.5,3.0,0.0,1.0,15.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10
1,2024-10-01T00:17:36,2024-10-01T00:26:22,1,2.2,1,N,162,145,1,11.4,3.5,0.5,3.3,0.0,1.0,19.7,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10


In [0]:
# Calculate trip duration in minutes for time-based analysis
df_trip = df_trip.withColumn(
    "trip_duration_minutes",
    (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 60
)
df_trip.limit(15).display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_pickup_date,trip_pickup_year,trip_pickup_month,trip_dropoff_date,trip_dropoff_year,trip_dropoff_month,trip_duration_minutes
2,2024-10-01T00:30:44,2024-10-01T00:48:26,1,3.0,1,N,162,246,1,18.4,1.0,0.5,1.5,0.0,1.0,24.9,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,17.7
1,2024-10-01T00:12:20,2024-10-01T00:25:25,1,2.2,1,N,48,236,1,14.2,3.5,0.5,3.8,0.0,1.0,23.0,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,13.083333333333334
1,2024-10-01T00:04:46,2024-10-01T00:13:52,1,2.7,1,N,142,24,1,13.5,3.5,0.5,3.7,0.0,1.0,22.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,9.1
1,2024-10-01T00:12:10,2024-10-01T00:23:01,1,3.1,1,N,233,75,1,14.2,3.5,0.5,2.0,0.0,1.0,21.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,10.85
1,2024-10-01T00:30:22,2024-10-01T00:30:39,1,0.0,1,N,262,262,3,3.0,3.5,0.5,0.0,0.0,1.0,8.0,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,0.2833333333333333
2,2024-10-01T00:31:20,2024-10-01T00:36:00,2,0.97,1,N,137,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,4.666666666666667
1,2024-10-01T00:42:57,2024-10-01T00:49:01,1,1.3,1,N,142,48,1,7.9,3.5,0.5,2.55,0.0,1.0,15.45,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,6.066666666666666
1,2024-10-01T00:59:55,2024-10-01T01:02:24,1,0.5,1,N,230,161,1,5.1,3.5,0.5,2.0,0.0,1.0,12.1,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,2.4833333333333334
1,2024-10-01T00:00:47,2024-10-01T00:04:22,0,1.1,1,N,142,237,1,7.2,3.5,0.5,3.0,0.0,1.0,15.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,3.5833333333333335
1,2024-10-01T00:17:36,2024-10-01T00:26:22,1,2.2,1,N,162,145,1,11.4,3.5,0.5,3.3,0.0,1.0,19.7,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,8.766666666666667


In [0]:
# Extract hour from pickup time and categorize into time slots
df_trip = df_trip.withColumn("pickup_hour", hour(col("tpep_pickup_datetime")))\
                 .withColumn("time_of_day",
                             when((col("pickup_hour") >= 6) & (col("pickup_hour") < 12), "Morning")\
                             .when((col("pickup_hour") >= 12) & (col("pickup_hour") < 18), "Afternoon")\
                             .when((col("pickup_hour") >= 18) & (col("pickup_hour") < 24), "Evening")\
                             .otherwise("Night"))
df_trip.limit(15).display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_pickup_date,trip_pickup_year,trip_pickup_month,trip_dropoff_date,trip_dropoff_year,trip_dropoff_month,trip_duration_minutes,pickup_hour,time_of_day
2,2024-10-01T00:30:44,2024-10-01T00:48:26,1,3.0,1,N,162,246,1,18.4,1.0,0.5,1.5,0.0,1.0,24.9,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,17.7,0,Night
1,2024-10-01T00:12:20,2024-10-01T00:25:25,1,2.2,1,N,48,236,1,14.2,3.5,0.5,3.8,0.0,1.0,23.0,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,13.083333333333334,0,Night
1,2024-10-01T00:04:46,2024-10-01T00:13:52,1,2.7,1,N,142,24,1,13.5,3.5,0.5,3.7,0.0,1.0,22.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,9.1,0,Night
1,2024-10-01T00:12:10,2024-10-01T00:23:01,1,3.1,1,N,233,75,1,14.2,3.5,0.5,2.0,0.0,1.0,21.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,10.85,0,Night
1,2024-10-01T00:30:22,2024-10-01T00:30:39,1,0.0,1,N,262,262,3,3.0,3.5,0.5,0.0,0.0,1.0,8.0,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,0.2833333333333333,0,Night
2,2024-10-01T00:31:20,2024-10-01T00:36:00,2,0.97,1,N,137,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,4.666666666666667,0,Night
1,2024-10-01T00:42:57,2024-10-01T00:49:01,1,1.3,1,N,142,48,1,7.9,3.5,0.5,2.55,0.0,1.0,15.45,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,6.066666666666666,0,Night
1,2024-10-01T00:59:55,2024-10-01T01:02:24,1,0.5,1,N,230,161,1,5.1,3.5,0.5,2.0,0.0,1.0,12.1,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,2.4833333333333334,0,Night
1,2024-10-01T00:00:47,2024-10-01T00:04:22,0,1.1,1,N,142,237,1,7.2,3.5,0.5,3.0,0.0,1.0,15.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,3.5833333333333335,0,Night
1,2024-10-01T00:17:36,2024-10-01T00:26:22,1,2.2,1,N,162,145,1,11.4,3.5,0.5,3.3,0.0,1.0,19.7,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,8.766666666666667,0,Night


In [0]:
# First check fare amount and passenger count ranges before setting validation rules
print("Fare Amount Statistics:")
df_trip.select(
    min("fare_amount").alias("min_fare"),
    max("fare_amount").alias("max_fare"),
    avg("fare_amount").alias("avg_fare"),
    stddev("fare_amount").alias("std_fare")
).show()

print("Passenger Count Distribution:")
df_trip.groupBy("passenger_count").count().orderBy("passenger_count").show(15)

Fare Amount Statistics:
+--------+---------+------------------+-----------------+
|min_fare| max_fare|          avg_fare|         std_fare|
+--------+---------+------------------+-----------------+
| -2261.2|335544.44|19.268509729966603|76.71984026978758|
+--------+---------+------------------+-----------------+

Passenger Count Distribution:
+---------------+--------+
|passenger_count|   count|
+---------------+--------+
|           NULL| 4091232|
|              0|  401354|
|              1|28632703|
|              2| 5410774|
|              3| 1282073|
|              4|  814889|
|              5|  320613|
|              6|  215798|
|              7|      56|
|              8|     192|
|              9|      36|
+---------------+--------+



In [0]:
# Validate fare amount and passenger count based on realistic NYC taxi ranges
df_trip = df_trip.withColumn("is_valid_fare",
                             when((col("fare_amount") >= 2.5) & (col("fare_amount") <= 200), True)\
                             .otherwise(False))\
                 .withColumn("is_valid_passenger_count",
                             when((col("passenger_count") >= 1) & (col("passenger_count") <= 6), True)\
                             .otherwise(False))
df_trip.limit(15).display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_pickup_date,trip_pickup_year,trip_pickup_month,trip_dropoff_date,trip_dropoff_year,trip_dropoff_month,trip_duration_minutes,pickup_hour,time_of_day,is_valid_fare,is_valid_passenger_count,tip_percentage,revenue_per_mile
2,2024-10-01T00:30:44,2024-10-01T00:48:26,1,3.0,1,N,162,246,1,18.4,1.0,0.5,1.5,0.0,1.0,24.9,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,17.7,0,Night,true,true,8.152173913043478,6.133333333333333
1,2024-10-01T00:12:20,2024-10-01T00:25:25,1,2.2,1,N,48,236,1,14.2,3.5,0.5,3.8,0.0,1.0,23.0,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,13.083333333333334,0,Night,true,true,26.76056338028169,6.454545454545454
1,2024-10-01T00:04:46,2024-10-01T00:13:52,1,2.7,1,N,142,24,1,13.5,3.5,0.5,3.7,0.0,1.0,22.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,9.1,0,Night,true,true,27.40740740740741,5.0
1,2024-10-01T00:12:10,2024-10-01T00:23:01,1,3.1,1,N,233,75,1,14.2,3.5,0.5,2.0,0.0,1.0,21.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,10.85,0,Night,true,true,14.084507042253522,4.580645161290322
1,2024-10-01T00:30:22,2024-10-01T00:30:39,1,0.0,1,N,262,262,3,3.0,3.5,0.5,0.0,0.0,1.0,8.0,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,0.2833333333333333,0,Night,true,true,0.0,0.0
2,2024-10-01T00:31:20,2024-10-01T00:36:00,2,0.97,1,N,137,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,4.666666666666667,0,Night,true,true,33.888888888888886,7.422680412371134
1,2024-10-01T00:42:57,2024-10-01T00:49:01,1,1.3,1,N,142,48,1,7.9,3.5,0.5,2.55,0.0,1.0,15.45,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,6.066666666666666,0,Night,true,true,32.27848101265822,6.076923076923077
1,2024-10-01T00:59:55,2024-10-01T01:02:24,1,0.5,1,N,230,161,1,5.1,3.5,0.5,2.0,0.0,1.0,12.1,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,2.4833333333333334,0,Night,true,true,39.21568627450981,10.2
1,2024-10-01T00:00:47,2024-10-01T00:04:22,0,1.1,1,N,142,237,1,7.2,3.5,0.5,3.0,0.0,1.0,15.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,3.5833333333333335,0,Night,true,false,41.666666666666664,6.545454545454545
1,2024-10-01T00:17:36,2024-10-01T00:26:22,1,2.2,1,N,162,145,1,11.4,3.5,0.5,3.3,0.0,1.0,19.7,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,8.766666666666667,0,Night,true,true,28.947368421052627,5.181818181818182


In [0]:
# Calculate business metrics: tip percentage and revenue per mile
df_trip = df_trip.withColumn("tip_percentage",
                             when(col("fare_amount") > 0, (col("tip_amount") / col("fare_amount")) * 100)\
                             .otherwise(0))\
                 .withColumn("revenue_per_mile",
                             when(col("trip_distance") > 0, col("fare_amount") / col("trip_distance"))\
                             .otherwise(0))
df_trip.limit(15).display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_pickup_date,trip_pickup_year,trip_pickup_month,trip_dropoff_date,trip_dropoff_year,trip_dropoff_month,trip_duration_minutes,pickup_hour,time_of_day,is_valid_fare,is_valid_passenger_count,tip_percentage,revenue_per_mile,data_quality_issue
2,2024-10-01T00:30:44,2024-10-01T00:48:26,1,3.0,1,N,162,246,1,18.4,1.0,0.5,1.5,0.0,1.0,24.9,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,17.7,0,Night,true,true,8.152173913043478,6.133333333333333,null
1,2024-10-01T00:12:20,2024-10-01T00:25:25,1,2.2,1,N,48,236,1,14.2,3.5,0.5,3.8,0.0,1.0,23.0,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,13.083333333333334,0,Night,true,true,26.76056338028169,6.454545454545454,null
1,2024-10-01T00:04:46,2024-10-01T00:13:52,1,2.7,1,N,142,24,1,13.5,3.5,0.5,3.7,0.0,1.0,22.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,9.1,0,Night,true,true,27.40740740740741,5.0,null
1,2024-10-01T00:12:10,2024-10-01T00:23:01,1,3.1,1,N,233,75,1,14.2,3.5,0.5,2.0,0.0,1.0,21.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,10.85,0,Night,true,true,14.084507042253522,4.580645161290322,null
1,2024-10-01T00:30:22,2024-10-01T00:30:39,1,0.0,1,N,262,262,3,3.0,3.5,0.5,0.0,0.0,1.0,8.0,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,0.2833333333333333,0,Night,true,true,0.0,0.0,null
2,2024-10-01T00:31:20,2024-10-01T00:36:00,2,0.97,1,N,137,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,4.666666666666667,0,Night,true,true,33.888888888888886,7.422680412371134,null
1,2024-10-01T00:42:57,2024-10-01T00:49:01,1,1.3,1,N,142,48,1,7.9,3.5,0.5,2.55,0.0,1.0,15.45,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,6.066666666666666,0,Night,true,true,32.27848101265822,6.076923076923077,null
1,2024-10-01T00:59:55,2024-10-01T01:02:24,1,0.5,1,N,230,161,1,5.1,3.5,0.5,2.0,0.0,1.0,12.1,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,2.4833333333333334,0,Night,true,true,39.21568627450981,10.2,null
1,2024-10-01T00:00:47,2024-10-01T00:04:22,0,1.1,1,N,142,237,1,7.2,3.5,0.5,3.0,0.0,1.0,15.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,3.5833333333333335,0,Night,true,false,41.666666666666664,6.545454545454545,Zero Passengers
1,2024-10-01T00:17:36,2024-10-01T00:26:22,1,2.2,1,N,162,145,1,11.4,3.5,0.5,3.3,0.0,1.0,19.7,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,8.766666666666667,0,Night,true,true,28.947368421052627,5.181818181818182,null


In [0]:
# Flag data quality issues for problematic records
df_trip = df_trip.withColumn("data_quality_issue",
                             when(col("fare_amount") < 0, "Negative Fare")\
                             .when(col("fare_amount") > 200, "Excessive Fare")\
                             .when(col("passenger_count").isNull(), "Missing Passenger Count")\
                             .when(col("passenger_count") == 0, "Zero Passengers")\
                             .when(col("passenger_count") > 6, "Excess Passengers")\
                             .otherwise(None))
df_trip.limit(15).display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_pickup_date,trip_pickup_year,trip_pickup_month,trip_dropoff_date,trip_dropoff_year,trip_dropoff_month,trip_duration_minutes,pickup_hour,time_of_day,is_valid_fare,is_valid_passenger_count,tip_percentage,revenue_per_mile,data_quality_issue
2,2024-10-01T00:30:44,2024-10-01T00:48:26,1,3.0,1,N,162,246,1,18.4,1.0,0.5,1.5,0.0,1.0,24.9,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,17.7,0,Night,true,true,8.152173913043478,6.133333333333333,null
1,2024-10-01T00:12:20,2024-10-01T00:25:25,1,2.2,1,N,48,236,1,14.2,3.5,0.5,3.8,0.0,1.0,23.0,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,13.083333333333334,0,Night,true,true,26.76056338028169,6.454545454545454,null
1,2024-10-01T00:04:46,2024-10-01T00:13:52,1,2.7,1,N,142,24,1,13.5,3.5,0.5,3.7,0.0,1.0,22.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,9.1,0,Night,true,true,27.40740740740741,5.0,null
1,2024-10-01T00:12:10,2024-10-01T00:23:01,1,3.1,1,N,233,75,1,14.2,3.5,0.5,2.0,0.0,1.0,21.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,10.85,0,Night,true,true,14.084507042253522,4.580645161290322,null
1,2024-10-01T00:30:22,2024-10-01T00:30:39,1,0.0,1,N,262,262,3,3.0,3.5,0.5,0.0,0.0,1.0,8.0,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,0.2833333333333333,0,Night,true,true,0.0,0.0,null
2,2024-10-01T00:31:20,2024-10-01T00:36:00,2,0.97,1,N,137,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,4.666666666666667,0,Night,true,true,33.888888888888886,7.422680412371134,null
1,2024-10-01T00:42:57,2024-10-01T00:49:01,1,1.3,1,N,142,48,1,7.9,3.5,0.5,2.55,0.0,1.0,15.45,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,6.066666666666666,0,Night,true,true,32.27848101265822,6.076923076923077,null
1,2024-10-01T00:59:55,2024-10-01T01:02:24,1,0.5,1,N,230,161,1,5.1,3.5,0.5,2.0,0.0,1.0,12.1,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,2.4833333333333334,0,Night,true,true,39.21568627450981,10.2,null
1,2024-10-01T00:00:47,2024-10-01T00:04:22,0,1.1,1,N,142,237,1,7.2,3.5,0.5,3.0,0.0,1.0,15.2,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,3.5833333333333335,0,Night,true,false,41.666666666666664,6.545454545454545,Zero Passengers
1,2024-10-01T00:17:36,2024-10-01T00:26:22,1,2.2,1,N,162,145,1,11.4,3.5,0.5,3.3,0.0,1.0,19.7,2.5,0.0,2024-10-01,2024,10,2024-10-01,2024,10,8.766666666666667,0,Night,true,true,28.947368421052627,5.181818181818182,null


In [0]:
df_trip.write.format('parquet')\
                  .mode('overwrite')\
                  .option("path","abfss://silver@nyctaxidataproject1.dfs.core.windows.net/trips2024data")\
                  .save()